In [ ]:
import numpy as np
import pandas as pd
import string

In [ ]:
df=pd.read_json('News_Category_Dataset_v2.json',lines=True)

In [ ]:
df

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
...,...,...,...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


In [ ]:
df=df.iloc[range(0,2000)]

In [ ]:
somevalues=['CRIME', 'ENTERTAINMENT', 'WORLD NEWS',  'POLITICS','SPORTS', 'BUSINESS', 'TRAVEL', 'MEDIA', 'TECH', 'SCIENCE']

In [ ]:
df=df.loc[df['category'].isin(somevalues)]

In [ ]:
df=df.reset_index(drop=True)

In [ ]:
def removePunct(txt):
  txt_nopunct="".join([c for c in txt if c not in string.punctuation])
  return txt_nopunct

In [ ]:
df['textClean']=df['short_description'].apply(lambda x:removePunct(x))
df

,category,headline,authors,link,short_description,date,textClean
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,She left her husband He killed their children ...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26,Of course it has a song
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26,The actor and his longtime girlfriend Anna Ebe...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26,The actor gives Dems an asskicking for not fig...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26,The Dietland actress said using the bags is a ...
...,...,...,...,...,...,...,...
1511,POLITICS,Senate Confirms Climate Change Denier To Lead ...,Chris D'Angelo,https://www.huffingtonpost.com/entry/jim-bride...,Rep. Jim Bridenstine (R-Okla.) will take over ...,2018-04-19,Rep Jim Bridenstine ROkla will take over contr...
1512,ENTERTAINMENT,Here Is Prince’s Original Version Of ‘Nothing ...,Elyse Wanshel,https://www.huffingtonpost.com/entry/heres-pri...,The Purple One’s estate released the song and ...,2018-04-19,The Purple One’s estate released the song and ...
1513,MEDIA,Laura Ingraham's Guest Mocks Her On Live TV Fo...,Ed Mazza,https://www.huffingtonpost.com/entry/laura-ing...,"""A high school kid shut you down!""",2018-04-19,A high school kid shut you down
1514,POLITICS,Former NYC Mayor Rudy Giuliani Is Joining Trum...,Ryan Grenoble,https://www.huffingtonpost.com/entry/rudy-giul...,The former prosecutor said he hopes to negotia...,2018-04-19,The former prosecutor said he hopes to negotia...


In [ ]:
df['category'].isnull().values.any()

False

In [ ]:
import re

In [ ]:
def tokenize(txt):
  tokens=re.split('\W+',txt)
  return tokens

In [ ]:
df['textCleanToken']=df['textClean'].apply(lambda x:tokenize(x.lower()))
df.head()

,category,headline,authors,link,short_description,date,textClean,textCleanToken
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,She left her husband He killed their children ...,"[she, left, her, husband, he, killed, their, c..."
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26,Of course it has a song,"[of, course, it, has, a, song]"
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26,The actor and his longtime girlfriend Anna Ebe...,"[the, actor, and, his, longtime, girlfriend, a..."
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26,The actor gives Dems an asskicking for not fig...,"[the, actor, gives, dems, an, asskicking, for,..."
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26,The Dietland actress said using the bags is a ...,"[the, dietland, actress, said, using, the, bag..."


In [ ]:
df.index

RangeIndex(start=0, stop=1516, step=1)

In [ ]:
import nltk
nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words('english')
stopwords[:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [ ]:
def removeStopwords(txt):
  txt_clean=[word for word in txt if word not in stopwords]
  return txt_clean


In [ ]:
df['noStopwords']=df['textCleanToken'].apply(lambda x: removeStopwords(x))
df.head()

,category,headline,authors,link,short_description,date,textClean,textCleanToken,noStopwords
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,She left her husband He killed their children ...,"[she, left, her, husband, he, killed, their, c...","[left, husband, killed, children, another, day..."
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26,Of course it has a song,"[of, course, it, has, a, song]","[course, song]"
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26,The actor and his longtime girlfriend Anna Ebe...,"[the, actor, and, his, longtime, girlfriend, a...","[actor, longtime, girlfriend, anna, eberstein,..."
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26,The actor gives Dems an asskicking for not fig...,"[the, actor, gives, dems, an, asskicking, for,...","[actor, gives, dems, asskicking, fighting, har..."
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26,The Dietland actress said using the bags is a ...,"[the, dietland, actress, said, using, the, bag...","[dietland, actress, said, using, bags, really,..."


In [ ]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()
def stemming(token_txt):
  text=[ps.stem(word) for word in token_txt]
  return text

In [ ]:
df['stemmed']=df['noStopwords'].apply(lambda x: stemming(x))
df.head()

,category,headline,authors,link,short_description,date,textClean,textCleanToken,noStopwords,stemmed
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,She left her husband He killed their children ...,"[she, left, her, husband, he, killed, their, c...","[left, husband, killed, children, another, day...","[left, husband, kill, children, anoth, day, am..."
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26,Of course it has a song,"[of, course, it, has, a, song]","[course, song]","[cours, song]"
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26,The actor and his longtime girlfriend Anna Ebe...,"[the, actor, and, his, longtime, girlfriend, a...","[actor, longtime, girlfriend, anna, eberstein,...","[actor, longtim, girlfriend, anna, eberstein, ..."
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26,The actor gives Dems an asskicking for not fig...,"[the, actor, gives, dems, an, asskicking, for,...","[actor, gives, dems, asskicking, fighting, har...","[actor, give, dem, asskick, fight, hard, enoug..."
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26,The Dietland actress said using the bags is a ...,"[the, dietland, actress, said, using, the, bag...","[dietland, actress, said, using, bags, really,...","[dietland, actress, said, use, bag, realli, ca..."


In [ ]:
def cleanWords(text):
  txt="".join([p for p in text if p not in string.punctuation])
  tokens=re.split("\W+",txt)
  txt=[ps.stem(word) for word in tokens if word not in stopwords]

In [ ]:
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))

In [ ]:
df['string']=df['stemmed'].apply(lambda x: listToString(x))
df.head()

,category,headline,authors,link,short_description,date,textClean,textCleanToken,noStopwords,stemmed,string
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,She left her husband He killed their children ...,"[she, left, her, husband, he, killed, their, c...","[left, husband, killed, children, another, day...","[left, husband, kill, children, anoth, day, am...",left husband kill children anoth day america
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26,Of course it has a song,"[of, course, it, has, a, song]","[course, song]","[cours, song]",cours song
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26,The actor and his longtime girlfriend Anna Ebe...,"[the, actor, and, his, longtime, girlfriend, a...","[actor, longtime, girlfriend, anna, eberstein,...","[actor, longtim, girlfriend, anna, eberstein, ...",actor longtim girlfriend anna eberstein tie kn...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26,The actor gives Dems an asskicking for not fig...,"[the, actor, gives, dems, an, asskicking, for,...","[actor, gives, dems, asskicking, fighting, har...","[actor, give, dem, asskick, fight, hard, enoug...",actor give dem asskick fight hard enough donal...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26,The Dietland actress said using the bags is a ...,"[the, dietland, actress, said, using, the, bag...","[dietland, actress, said, using, bags, really,...","[dietland, actress, said, use, bag, realli, ca...",dietland actress said use bag realli cathart t...


In [ ]:
def removeNum(txt):
  result = ''.join([i for i in txt if not i.isdigit()])
  return result

In [ ]:
df['Nonum']=df['string'].apply(lambda x: removeNum(x))

In [ ]:
df.head()

,category,headline,authors,link,short_description,date,textClean,textCleanToken,noStopwords,stemmed,string,Nonum
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,She left her husband He killed their children ...,"[she, left, her, husband, he, killed, their, c...","[left, husband, killed, children, another, day...","[left, husband, kill, children, anoth, day, am...",left husband kill children anoth day america,left husband kill children anoth day america
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26,Of course it has a song,"[of, course, it, has, a, song]","[course, song]","[cours, song]",cours song,cours song
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26,The actor and his longtime girlfriend Anna Ebe...,"[the, actor, and, his, longtime, girlfriend, a...","[actor, longtime, girlfriend, anna, eberstein,...","[actor, longtim, girlfriend, anna, eberstein, ...",actor longtim girlfriend anna eberstein tie kn...,actor longtim girlfriend anna eberstein tie kn...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26,The actor gives Dems an asskicking for not fig...,"[the, actor, gives, dems, an, asskicking, for,...","[actor, gives, dems, asskicking, fighting, har...","[actor, give, dem, asskick, fight, hard, enoug...",actor give dem asskick fight hard enough donal...,actor give dem asskick fight hard enough donal...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26,The Dietland actress said using the bags is a ...,"[the, dietland, actress, said, using, the, bag...","[dietland, actress, said, using, bags, really,...","[dietland, actress, said, use, bag, realli, ca...",dietland actress said use bag realli cathart t...,dietland actress said use bag realli cathart t...


##CountVec

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
X=cv.fit_transform(df['Nonum'])
print(X.shape)

(1516, 4112)


In [ ]:
df2=pd.DataFrame(X.toarray(),columns=cv.get_feature_names())
df2.head()

,aaron,abandon,abbott,abc,abdeslam,abdul,abdulfatai,abel,abil,abl,abnorm,aboard,abort,abram,abroad,abrupt,absolut,abus,academi,accept,access,accid,accord,account,accus,acknowledg,aclu,acquir,acquit,across,act,action,actionpack,activ,activist,actor,actress,actual,ad,ada,...,written,wrong,wrongli,wrote,wyli,xenophob,xfile,ximena,xray,yall,yanni,ye,yeah,year,yearold,yeezi,yehuda,yell,yellowston,yesterday,yet,yike,york,yorker,youd,youll,young,younger,your,youtub,yu,zachari,zeitgeist,zimmerman,zing,zink,zionist,zoo,zte,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
scorep1=X.nnz / float(X.shape[0])
scorep1

8.362137203166228

In [ ]:
scorep1/float(X.shape[1])

0.0020335936778127987

##TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv=TfidfVectorizer()
X=tv.fit_transform(df['Nonum'])
print(X.shape)

(1516, 4112)


In [ ]:
df3=pd.DataFrame(X.toarray(),columns=tv.get_feature_names())

In [ ]:
df3.head()

,aaron,abandon,abbott,abc,abdeslam,abdul,abdulfatai,abel,abil,abl,abnorm,aboard,abort,abram,abroad,abrupt,absolut,abus,academi,accept,access,accid,accord,account,accus,acknowledg,aclu,acquir,acquit,across,act,action,actionpack,activ,activist,actor,actress,actual,ad,ada,...,written,wrong,wrongli,wrote,wyli,xenophob,xfile,ximena,xray,yall,yanni,ye,yeah,year,yearold,yeezi,yehuda,yell,yellowston,yesterday,yet,yike,york,yorker,youd,youll,young,younger,your,youtub,yu,zachari,zeitgeist,zimmerman,zing,zink,zionist,zoo,zte,zuckerberg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.252427,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.286055,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.284459,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
scorep1=X.nnz / float(X.shape[0])
scorep1

8.362137203166228

##Doc2Vec

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:

from nltk.tokenize import word_tokenize

In [ ]:
import nltk


nltk.download('punkt')
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(df['Nonum'])]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tagged_data

In [ ]:
max_epochs = 100
vec_size = 200
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [ ]:
print(model.docvecs['1'])

[-0.22836636 -0.05085429  0.4864856   0.22945361  0.10682665  0.06709168
  0.1071208   0.2329128  -0.06672534 -0.02953051 -0.44694775 -0.09687547
  0.09679859  0.19796368  0.03407633  0.26432946 -0.34870514 -0.3245653
 -0.16854298 -0.1336786   0.15044025 -0.15501872  0.1903121  -0.41987863
 -0.04142734 -0.10135697  0.04073799 -0.31465054  0.16702896 -0.04193965
  0.36903787  0.06868266 -0.14450087  0.09105323 -0.38935176 -0.70872635
 -0.1746882  -0.2229578  -0.02686821 -0.19354206  0.41538933 -0.42928788
  0.414149   -0.19773607  0.5050463  -0.09136776  0.01642741 -0.07134244
  0.37403935  0.3967638   0.2091183   0.22726555  0.36061683  0.0731743
  0.16010825 -0.36941198 -0.27505517 -0.07903215 -0.17480054 -0.19337678
  0.0399317   0.5225633  -0.18825468 -0.14082329  0.15657933 -0.28336996
 -0.12899405  0.20450796 -0.4281351   0.2542332   0.13337654 -0.17715366
 -0.04279566  0.03851324  0.32277903  0.055605   -0.59731543 -0.08897056
 -0.11371558  0.22782318  0.02141695  0.0386331  -0.1

In [ ]:
len(model.docvecs['5'])

200

In [ ]:
model.docvecs['5']

array([-0.28563774, -0.25545   , -0.12138844,  0.28918967, -0.08777292,
        0.14029808,  0.25538895,  0.5858257 , -0.29318878, -0.00697948,
       -0.26221383,  0.4110887 ,  0.6545507 , -0.17006163,  0.43236542,
       -0.61407524,  0.06622802, -0.3109174 , -0.3625914 , -0.76323897,
        0.7160523 , -0.4316496 ,  0.40266424, -0.5447584 ,  0.97862726,
       -0.24402137, -0.3033918 , -0.11988214, -0.20077835,  0.35246748,
        0.21106349,  0.335415  , -0.32605323, -0.18399024, -0.48606732,
       -0.622794  , -0.05389021, -0.03014844, -0.05462959, -0.0114894 ,
        0.47885993, -0.21240345, -0.18561915,  0.46344015, -0.19460122,
       -0.07348348, -0.70349866, -0.9683407 ,  0.07479791,  0.5270473 ,
       -0.12112259,  0.8676931 ,  0.76848507,  0.17607975,  0.12070855,
       -0.34848908, -0.2820241 ,  0.67620933,  0.08554089, -0.7217774 ,
        0.2875459 ,  0.3159251 ,  0.15554975,  0.43177438,  0.56957966,
       -0.12747265,  0.25857976, -0.54087275,  0.12196622,  0.22

In [ ]:
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)

[('1512', 0.7949838638305664), ('1157', 0.7656614184379578), ('1155', 0.7439842224121094), ('1319', 0.7220793962478638), ('130', 0.6981788873672485), ('1174', 0.6902753114700317), ('284', 0.6863356828689575), ('1110', 0.6849075555801392), ('1513', 0.6832395792007446), ('978', 0.681418776512146)]


In [ ]:
similar_doc2 = model.docvecs.most_similar('2')
print(similar_doc2)

[('284', 0.7082858085632324), ('31', 0.6945168972015381), ('908', 0.678748607635498), ('871', 0.667586088180542), ('148', 0.6565464735031128), ('955', 0.6470935940742493), ('497', 0.6353871822357178), ('954', 0.6346310377120972), ('273', 0.6284869909286499), ('629', 0.6278048157691956)]


##Random Forest

In [ ]:
df

,category,headline,authors,link,short_description,date,textClean,textCleanToken,noStopwords,stemmed,string,Nonum
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,She left her husband He killed their children ...,"[she, left, her, husband, he, killed, their, c...","[left, husband, killed, children, another, day...","[left, husband, kill, children, anoth, day, am...",left husband kill children anoth day america,left husband kill children anoth day america
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26,Of course it has a song,"[of, course, it, has, a, song]","[course, song]","[cours, song]",cours song,cours song
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26,The actor and his longtime girlfriend Anna Ebe...,"[the, actor, and, his, longtime, girlfriend, a...","[actor, longtime, girlfriend, anna, eberstein,...","[actor, longtim, girlfriend, anna, eberstein, ...",actor longtim girlfriend anna eberstein tie kn...,actor longtim girlfriend anna eberstein tie kn...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26,The actor gives Dems an asskicking for not fig...,"[the, actor, gives, dems, an, asskicking, for,...","[actor, gives, dems, asskicking, fighting, har...","[actor, give, dem, asskick, fight, hard, enoug...",actor give dem asskick fight hard enough donal...,actor give dem asskick fight hard enough donal...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26,The Dietland actress said using the bags is a ...,"[the, dietland, actress, said, using, the, bag...","[dietland, actress, said, using, bags, really,...","[dietland, actress, said, use, bag, realli, ca...",dietland actress said use bag realli cathart t...,dietland actress said use bag realli cathart t...
...,...,...,...,...,...,...,...,...,...,...,...,...
1511,POLITICS,Senate Confirms Climate Change Denier To Lead ...,Chris D'Angelo,https://www.huffingtonpost.com/entry/jim-bride...,Rep. Jim Bridenstine (R-Okla.) will take over ...,2018-04-19,Rep Jim Bridenstine ROkla will take over contr...,"[rep, jim, bridenstine, rokla, will, take, ove...","[rep, jim, bridenstine, rokla, take, control, ...","[rep, jim, bridenstin, rokla, take, control, s...",rep jim bridenstin rokla take control space ag...,rep jim bridenstin rokla take control space ag...
1512,ENTERTAINMENT,Here Is Prince’s Original Version Of ‘Nothing ...,Elyse Wanshel,https://www.huffingtonpost.com/entry/heres-pri...,The Purple One’s estate released the song and ...,2018-04-19,The Purple One’s estate released the song and ...,"[the, purple, one, s, estate, released, the, s...","[purple, one, estate, released, song, previous...","[purpl, one, estat, releas, song, previous, un...",purpl one estat releas song previous unseen fo...,purpl one estat releas song previous unseen fo...
1513,MEDIA,Laura Ingraham's Guest Mocks Her On Live TV Fo...,Ed Mazza,https://www.huffingtonpost.com/entry/laura-ing...,"""A high school kid shut you down!""",2018-04-19,A high school kid shut you down,"[a, high, school, kid, shut, you, down]","[high, school, kid, shut]","[high, school, kid, shut]",high school kid shut,high school kid shut
1514,POLITICS,Former NYC Mayor Rudy Giuliani Is Joining Trum...,Ryan Grenoble,https://www.huffingtonpost.com/entry/rudy-giul...,The former prosecutor said he hopes to negotia...,2018-04-19,The former prosecutor said he hopes to negotia...,"[the, former, 

In [ ]:
df2=df2.dropna()

In [ ]:
df3=df3.dropna()

In [ ]:
df['category']

0               CRIME
1       ENTERTAINMENT
2       ENTERTAINMENT
3       ENTERTAINMENT
4       ENTERTAINMENT
            ...      
1511         POLITICS
1512    ENTERTAINMENT
1513            MEDIA
1514         POLITICS
1515            MEDIA
Name: category, Length: 1516, dtype: object

In [ ]:
df2['category']=df['category']

In [ ]:
df2['category'].isnull().values.any()

False

In [ ]:
df3['category']=df['category']

In [ ]:
df3

,aaron,abandon,abbott,abc,abdeslam,abdul,abdulfatai,abel,abil,abl,abnorm,aboard,abort,abram,abroad,abrupt,absolut,abus,academi,accept,access,accid,accord,account,accus,acknowledg,aclu,acquir,acquit,across,act,action,actionpack,activ,activist,actor,actress,actual,ad,ada,...,wrong,wrongli,wrote,wyli,xenophob,xfile,ximena,xray,yall,yanni,ye,yeah,year,yearold,yeezi,yehuda,yell,yellowston,yesterday,yet,yike,york,yorker,youd,youll,young,younger,your,youtub,yu,zachari,zeitgeist,zimmerman,zing,zink,zionist,zoo,zte,zuckerberg,category
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CRIME
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ENTERTAINMENT
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.252427,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ENTERTAINMENT
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.286055,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ENTERTAINMENT
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.284459,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ENTERTAINMENT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POLITICS
1512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ENTERTAINMENT
1513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEDIA
1514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POLITICS


##RandomForest

###Countvector

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df2.drop(['category'],axis='columns'),df2.category,test_size=0.2)

In [ ]:
rfcmodel=RandomForestClassifier()
rfcmodel.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
x_train

,aaron,abandon,abbott,abc,abdeslam,abdul,abdulfatai,abel,abil,abl,abnorm,aboard,abort,abram,abroad,abrupt,absolut,abus,academi,accept,access,accid,accord,account,accus,acknowledg,aclu,acquir,acquit,across,act,action,actionpack,activ,activist,actor,actress,actual,ad,ada,...,written,wrong,wrongli,wrote,wyli,xenophob,xfile,ximena,xray,yall,yanni,ye,yeah,year,yearold,yeezi,yehuda,yell,yellowston,yesterday,yet,yike,york,yorker,youd,youll,young,younger,your,youtub,yu,zachari,zeitgeist,zimmerman,zing,zink,zionist,zoo,zte,zuckerberg
144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1221,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
505,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1180,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
808,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
732,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
rfcmodel.score(x_test,y_test)

0.6381578947368421

###TF-IDF

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df3.drop(['category'],axis='columns'),df3.category,test_size=0.2)

In [ ]:
rfcmodel=RandomForestClassifier()
rfcmodel.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
rfcmodel.score(x_test,y_test)

0.6546052631578947

###Doc2Vec

In [ ]:
nnvec=model.docvecs.vectors_docs

In [ ]:
nnvec

array([[-0.7020635 , -0.21735816,  0.65339226, ...,  0.16823867,
         0.66061896, -0.26348144],
       [-0.22836636, -0.05085429,  0.4864856 , ..., -0.07064036,
         0.40783513, -0.17327423],
       [-0.6456676 ,  0.7481331 ,  0.07275004, ..., -0.00271349,
         0.6849933 , -0.4851325 ],
       ...,
       [-0.29574135,  0.01789534,  0.23221704, ...,  0.27840847,
         0.5366604 , -0.14767376],
       [-0.25547394,  0.16573495,  0.59997386, ..., -0.11428248,
         0.58236444, -0.29929525],
       [-0.15575422,  0.4568583 ,  0.95024854, ...,  0.03646578,
         0.14660577, -0.42557314]], dtype=float32)

In [ ]:
nnvecdf = pd.DataFrame(nnvec)

In [ ]:
nnvecdf['category'] = df['category']

In [ ]:
nnvecdf['category'].isnull().values.any()

False

In [ ]:
nnvecdf=nnvecdf.dropna()

In [ ]:
nnvecdf['category'].isnull().values.any()

False

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(nnvecdf.drop(['category'],axis='columns'),nnvecdf.category,test_size=0.2)

In [ ]:
rfcmodel=RandomForestClassifier()
rfcmodel.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
rfcmodel.score(x_test,y_test)

0.5888157894736842